In [1]:
# read data

import pandas as pd

df_practice = pd.read_json("./data/practice_data/original_data/practice.json", lines=True, dtype={'num': str})
df_practice

,ID,url,claim,posted,sci_digest,justification,issues,image_data,label,evidence
0,FMD_practice_0,https://www.snopes.com/fact-check/wegmans-face...,Wegmans $80 Coupon Scam,02/01/2016,[Grocery chain Wegmans warned Facebook users t...,"In late January 2016, Facebook users began sha...",[credit],[{'image_src': 'https://drive.google.com/uc?ex...,False,A Better Business Bureauarticle provided shopp...
1,FMD_practice_1,https://www.snopes.com/fact-check/hawker-beech...,Seller of Beechcraft aircraft,01/02/2012,[Did the U.S. government unfairly exclude Hawk...,Claim: The U.S. government unfairly excluded ...,[taxes],[],False,The Air Force maintains that the process was f...
2,FMD_practice_2,https://www.politifact.com/factchecks/2016/apr...,"In the 2014 election, 80 percentofpoor people ...",04/24/2016,[],Income inequality is the core issue of Bernie ...,"[National, Income, Voting Record]",[{'image_src': 'https://drive.google.com/uc?ex...,True,"Well, because poor people dont vote. I mean, t..."
3,FMD_practice_3,https://www.snopes.com/fact-check/snapchat-fac...,Is Snapchat creating a database that uses faci...,07/26/2017,[Online conspiracy theorists claim Snapchat's ...,One of the more whimsical messaging options of...,[accountability],[{'image_src': 'https://drive.google.com/uc?ex...,False,"TEENWOLF (@TEENWOLFREMIX) October 3, 2015It w..."
4,FMD_practice_4,https://www.politifact.com/factchecks/2012/aug...,Says About 7 in 10 of Oregons inmates are in p...,08/03/2012,[],"Frankly, PolitiFact Oregon was surprised to sp...","[Oregon, Criminal Justice, Crime, Public Safet...",[],True,"Frankly, PolitiFact Oregon was surprised to sp..."
...,...,...,...,...,...,...,...,...,...,...
595,FMD_practice_595,https://www.politifact.com/factchecks/2016/jul...,We have the most productive workers in the world.,07/27/2016,[],On the third night of the Democratic conventio...,"[National, Economy, Workers]",[],True,When the OECDcompares the GDP per hour workeda...
596,FMD_practice_596,https://www.snopes.com/fact-check/the-loan-arr...,Obama mandated that banks provide loans to ind...,10/03/2008,[Barack Obama filed a lawsuit to require banks...,Claim: Barack Obama filed a lawsuit to requir...,[loan],[],NEI,The 1994 case of Buycks-Roberson v. Citibank F...
597,FMD_practice_597,https://www.snopes.com/fact-check/secret-disse...,Secret Service Coverage of John Kerry,10/02/2004,[Would the Secret Service have to provide life...,"Claim: If John Kerry were elected President, ...",[asset],[],False,"John Kerry and his wife, Teresa Heinz Kerry, t..."
598,FMD_practice_598,https://www.politifact.com/factchecks/2015/sep...,"According to reports, Ronald Reagan rescinded ...",09/25/2015,[],Stephen Colbert has intervieweda slew ofpresid...,"[Taxes, PunditFact]",[{'image_src': 'https://lh3.googleusercontent....,True,PolitiFact explored Cruzs point about economic...


In [2]:
# split data
from sklearn.model_selection import train_test_split

json_trainval, json_test = train_test_split(df_practice, test_size=50, random_state=42)
json_train, json_val = train_test_split(json_trainval, test_size=100, random_state=42)
len(json_train),len(json_val),len(json_test)

(450, 100, 50)

In [3]:
# construct instruction train data

import pandas as pd
import random
import json

data_type = "train"

input_path = "./data/practice_data/instruct_data/"


df = json_train

pre_instructs = ['''Please determine whether the claim is True, False, or Not Enough Information (NEI) based on contextual information, and provide an appropriate explanation.
The answer need to use the following format:
Prediction: [True, or False, or NEI]
Explanation: [Explain why the above prediction was made]
''']

def constuct_fullinstract(x, y, ID):
    
    pre_instruct = pre_instructs[0]
    Instruction = "Task: " + pre_instruct + " Claim: " + x + " " + "\ncontextual information: " + y
    
    return Instruction

def constuct_answer(x, y, ID):
    
    answer = "Prediction: " + x + ".\nExplanation: " + y
    return answer

df['Instruction'] = df.apply(lambda row: constuct_fullinstract(row['claim'], row["justification"],row["ID"]), axis=1)
df['Answer'] = df.apply(lambda row: constuct_answer(row['label'], row["evidence"],row["ID"]), axis=1)

with open(input_path + './FMD_%s.json'%(data_type), 'w',encoding="utf-8") as write_f:
    
    for index, row in df.iterrows():
        data_one = {}
        
        data_one["instruction"] = row["Instruction"]
        data_one["input"] = ""
        data_one["output"] = row["Answer"]

        write_f.write(json.dumps(data_one, indent=None, ensure_ascii=False) + "\n")


In [4]:
# construct instruction val data

import pandas as pd
import random
import json

data_type = "val"

input_path = "./data/practice_data/instruct_data/"


df = json_val

pre_instructs = ['''Please determine whether the claim is True, False, or Not Enough Information (NEI) based on contextual information, and provide an appropriate explanation.
The answer need to use the following format:
Prediction: [True, or False, or NEI]
Explanation: [Explain why the above prediction was made]
''']

def constuct_fullinstract(x, y, ID):
    
    pre_instruct = pre_instructs[0]
    Instruction = "Task: " + pre_instruct + " Claim: " + x + " " + "\ncontextual information: " + y
    
    return Instruction

def constuct_answer(x, y, ID):
    
    answer = "Prediction: " + x + ".\nExplanation: " + y
    return answer

df['Instruction'] = df.apply(lambda row: constuct_fullinstract(row['claim'], row["justification"],row["ID"]), axis=1)
df['Answer'] = df.apply(lambda row: constuct_answer(row['label'], row["evidence"],row["ID"]), axis=1)

with open(input_path + './FMD_%s.json'%(data_type), 'w',encoding="utf-8") as write_f:
    
    for index, row in df.iterrows():
        data_one = {}
        
        data_one["instruction"] = row["Instruction"]
        data_one["input"] = ""
        data_one["output"] = row["Answer"]

        write_f.write(json.dumps(data_one, indent=None, ensure_ascii=False) + "\n")


In [5]:
# construct instruction test data

import pandas as pd
import random
import json

data_type = "test"

input_path = "./data/practice_data/instruct_data/"


df = json_test

pre_instructs = ['''Please determine whether the claim is True, False, or Not Enough Information (NEI) based on contextual information, and provide an appropriate explanation.
The answer need to use the following format:
Prediction: [True, or False, or NEI]
Explanation: [Explain why the above prediction was made]
''']

def constuct_fullinstract(x, y, ID):
    
    pre_instruct = pre_instructs[0]
    Instruction = "Task: " + pre_instruct + " Claim: " + x + " " + "\ncontextual information: " + y
    
    return Instruction

def constuct_answer(x, y, ID):
    
    answer = "Prediction: " + x + ".\nExplanation: " + y
    return answer

df['Instruction'] = df.apply(lambda row: constuct_fullinstract(row['claim'], row["justification"],row["ID"]), axis=1)
df['Answer'] = df.apply(lambda row: constuct_answer(row['label'], row["evidence"],row["ID"]), axis=1)

with open(input_path + './FMD_%s.json'%(data_type), 'w',encoding="utf-8") as write_f:
    
    for index, row in df.iterrows():
        data_one = {}
        
        data_one["instruction"] = row["Instruction"]
        data_one["input"] = ""
        data_one["output"] = row["Answer"]

        write_f.write(json.dumps(data_one, indent=None, ensure_ascii=False) + "\n")
